In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Nov 23 14:42:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/Cattle_Identification"

from google.colab import drive
drive.mount('/content/drive')
import sys
import os
sys.path.insert(0, notebook_dir)
os.chdir(notebook_dir)

Mounted at /content/drive


In [3]:
import requests
symconv = "https://github.com/pwang697/Cattle_Identification/blob/main/symconv.py?raw=true"
r_s = requests.get(symconv, allow_redirects=True)
with open('symconv.py', 'wb') as f:
    f.write(r_s.content)

import numpy as np
import symconv as sc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data.sampler import BatchSampler
from itertools import combinations
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
def feature_label_split(npz):
  import numpy as np
  import torch
  X_array = npz['images']
  y_array = npz['labels']
  state = np.random.get_state()
  np.random.shuffle(X_array)
  np.random.set_state(state)
  np.random.shuffle(y_array)
  return torch.tensor(np.transpose(X_array, (0,3,1,2))).float(), torch.tensor(y_array).float()

In [5]:
X_train, y_train = feature_label_split(np.load(notebook_dir + '/datasets/train.npz'))
X_test, y_test = feature_label_split(np.load(notebook_dir + '/datasets/test.npz'))
print(f"Image train shape: {X_train.shape}")
print(f"Label train shape: {y_train.shape}")
print(f"Image test shape: {X_test.shape}")
print(f"Label test shape: {y_test.shape}")

Image train shape: torch.Size([826, 3, 1000, 1000])
Label train shape: torch.Size([826])
Image test shape: torch.Size([182, 3, 1000, 1000])
Label test shape: torch.Size([182])


In [6]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [7]:
class BalancedBatchSampler(BatchSampler):
    """
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels))
        self.label_to_indices = {label: np.where(  np.array(self.labels) == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

In [8]:
class RandomTripletSelector():
    """
    Select random negative example for each positive pair to create triplets
    """

    def __init__(self):
        super(RandomTripletSelector, self).__init__()

    def get_triplets(self, embeddings, labels):
        labels = labels.cpu().data.numpy()
        triplets = []
        for label in set(labels):
            label_mask = (labels == label)
            label_indices = np.where(label_mask)[0]
            if len(label_indices) < 2:
                continue
            negative_indices = np.where(np.logical_not(label_mask))[0]
            anchor_positives = list(combinations(label_indices, 2))  # All anchor-positive pairs

            # random choose one negative example for each positive pair
            temp_triplets = [[anchor_positive[0], anchor_positive[1], np.random.choice(negative_indices)] for anchor_positive in anchor_positives]
            triplets += temp_triplets

        return torch.LongTensor(np.array(triplets))

In [9]:
class TripletLoss(nn.Module):
    """
    Triplets loss
    Takes a batch of embeddings and corresponding labels.
    Triplets are generated using triplet_selector object that take embeddings and targets and return indices of
    triplets
    """

    def __init__(self, margin, triplet_selector):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.triplet_selector = triplet_selector

    def forward(self, embeddings, target):

        triplets = self.triplet_selector.get_triplets(embeddings, target)

        if embeddings.is_cuda:
            triplets = triplets.cuda()


        anchor_idx= triplets[:, 0]
        positive_idx= triplets[:, 1]
        negative_idx= triplets[:, 2]


        ap_distances = (embeddings[anchor_idx] - embeddings[positive_idx]).pow(2).sum(1)
        an_distances = (embeddings[anchor_idx] - embeddings[negative_idx]).pow(2).sum(1)
        losses = F.relu(ap_distances - an_distances + self.margin)

        return losses.mean()

In [10]:
class Trainer():
    def __init__(self,
                 model: torch.nn.Module,
                 device: torch.device,
                 criterion: torch.nn.Module,
                 optimizer: torch.optim.Optimizer,
                 training_DataLoader: torch.utils.data.Dataset,
                 test_DataLoader: torch.utils.data.Dataset,
                 epochs: int
                 ):

        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.training_DataLoader = training_DataLoader
        self.test_DataLoader = test_DataLoader
        self.device = device
        self.epochs = epochs


    def run_trainer(self):

      train_loss = []
      test_loss = []

      for epoch in tqdm(range(self.epochs)):
          self.model.train()  # train mode

          train_losses=[]
          for batch in self.training_DataLoader:

              x,y=batch
              input, target = x.to(self.device), y.to(self.device)  # send to device (GPU or CPU)
              self.optimizer.zero_grad()  # zerograd the parameters
              out = self.model(input)  # one forward pass
              loss = self.criterion(out, target)  # calculate loss

              loss_value = loss.item()
              train_losses.append(loss_value)

              loss.backward()  # one backward pass
              self.optimizer.step()  # update the parameters



          self.model.eval()  # evaluation mode
          test_losses = []  # accumulate the losses here

          for batch in self.test_DataLoader:

              x,y=batch
              input, target = x.to(self.device), y.to(self.device)  # send to device (GPU or CPU)

              with torch.no_grad():
                  out = self.model(input)   # one forward pass
                  loss = self.criterion(out, target) # calculate loss

                  loss_value = loss.item()
                  test_losses.append(loss_value)

          mean_train_loss = np.mean(train_losses)
          mean_test_loss = np.mean(test_losses)
          train_loss.append(mean_train_loss)
          test_loss.append(mean_test_loss)

          # print the results
          print(f'EPOCH: {epoch+1:0>{len(str(self.epochs))}}/{self.epochs}',end=' ')
          print(f'TRAIN-LOSS: {mean_train_loss:.4f}',end=' ')
          print(f'TEST-LOSS: {mean_train_loss:.4f}',end='\n')

      plt.figure(figsize=(8,6))
      plt.title('Loss curve')
      plt.plot(range(self.epochs), train_loss, label="train_loss")
      plt.plot(range(self.epochs), test_loss, label="test_loss")
      plt.legend()
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.show

In [11]:
class SymConvNet(nn.Module):
  def __init__(self):
      super(SymConvNet, self).__init__()

      self.conv1 = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=5, stride=5),
      ) # (3,1000,1000) -> (16,200,200)

      self.conv2 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=5, stride=5),
      ) # (16,200,200) -> (32,40,40)

      self.conv3 = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
      ) # (32,40,40) -> (64,20,20)

      self.conv4 = nn.Sequential(
          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
      ) # (64,20,20) -> (128,10,10)

      self.sconv = nn.Sequential(
          sc.Slice(rotation=2,reflection=False),
          sc.SymmetryConv2d(in_filters=128,out_filters=256,kernel_size=10,stride=10,rotation=2,reflection=False),
          nn.ReLU(inplace=True),
          sc.SymmetryPool(),
      ) # (128,10,10) -> (256,1,1)

      #Dense layer block
      self.dense = nn.Sequential(
          nn.Linear(256,256),
          nn.Dropout(p=0.1,inplace=True),
          nn.Softmax(dim=0),
          nn.Linear(256,91),
      )

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.sconv(x)
    x = x.view(x.size()[0], -1)
    x = self.dense(x)
    return x

In [12]:
#parameters
learning_rate = 1e-4
epochs = 50
n_classes = 91
n_samples = 5
margin = 0.01

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device=torch.device('cpu')

# model
model = SymConvNet().to(device)

# dataloaders
train_batch_sampler = BalancedBatchSampler(y_train.tolist(), n_classes=n_classes, n_samples=n_samples)
test_batch_sampler = BalancedBatchSampler(y_test.tolist(), n_classes=n_classes, n_samples=n_samples)

triplets_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
triplets_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)


# criterion
criterion = TripletLoss(margin,  RandomTripletSelector())

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=triplets_train_loader,
                  test_DataLoader=triplets_test_loader,
                  epochs=epochs)

# start training
trainer.run_trainer()

  0%|          | 0/50 [00:02<?, ?it/s]


OutOfMemoryError: ignored